In [ ]:
import os
import re
import sys
import glob
import pickle
import tables
from scipy.fft import fft, fftfreq
from scipy.signal import butter, filtfilt, lombscargle
from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib notebook

import tensorflow as tf
from tensorflow import keras

if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments
from dlml.data import read_area_values, load_data_areas, load_data_slide
from dlml.nn import predict, DownSampling1D, SpectralPooling, MaxPooling1DWithArgmax

#### Find the best experiment given a set of tags

In [ ]:
area_ID = 1
area_measure = 'momentum'
stoch_load_bus_IDs = []
rec_bus_IDs = [3]
H_G1, D, DZA = None, None, None # 500, 2, 0
additional_tags = ['ReLU_none', 'converted_from_PowerFactory', 'all_stoch_loads', 'data_subset']
missing_tags = []
use_FFT = False
if use_FFT:
    additional_tags.append('fft')
else:
    missing_tags.append('fft')
pooling_type = 'argmax'
if pooling_type is not None and pooling_type != '':
    additional_tags.append(pooling_type + '_pooling')

# training on frequency data, 2 output values
# experiment_ID = '9ea493c789b542bf979c51a6031f4044'
# training on frequency data, 4 output values
# experiment_ID = 'f6d9a03f1cfe450288e9cb86da94235f'
# training on time series data, 2 output values
# experiment_ID = '034a1edb0797475b985f0e1335dab383'
# training on time series data, 4 output values
# experiment_ID = 'b346a89d384c4db2ba4058a2c83c4f12'
experiment_ID = None

if experiment_ID is not None:
    from comet_ml.api import API, APIExperiment
    api = API(api_key = os.environ['COMET_API_KEY'])
    experiment = api.get_experiment('danielelinaro', 'inertia', experiment_ID)
    sys.stdout.write(f'Getting metrics for experiment {experiment_ID[:6]}... ')
    sys.stdout.flush()
    metrics = experiment.get_metrics()
    sys.stdout.write('done.\n')
    val_loss = []
    for m in metrics:
        if m['metricName'] == 'val_loss':
            val_loss.append(float(m['metricValue']))
        elif m['metricName'] == 'mape_prediction':
            MAPE = float(m['metricValue'])
    val_loss = np.array(val_loss)
else:
    # find the best experiment that matches the set of tags above
    experiments = collect_experiments(area_ID, area_measure=area_measure, D=D, DZA=DZA, \
                                      stoch_load_bus_IDs=stoch_load_bus_IDs, H_G1=H_G1, \
                                      rec_bus_IDs=rec_bus_IDs, additional_tags=additional_tags, \
                                      missing_tags=missing_tags, verbose=True)
    experiment_IDs = list(experiments.keys())
    experiment_ID = experiment_IDs[np.argmin([expt['val_loss'].min() for expt in experiments.values()])]
    experiment_ID = experiment_IDs[np.argmin([expt['MAPE'] for expt in experiments.values()])]

    MAPE = experiments[experiment_ID]['MAPE']
    loss = experiments[experiment_ID]['loss']
    val_loss = experiments[experiment_ID]['val_loss']
    batch_loss = experiments[experiment_ID]['batch_loss']
    tags = experiments[experiment_ID]['tags']
print(f'Selected experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE:.4f}%).')

#### Load the model

In [ ]:
experiments_path = '../experiments/neural_network/'
network_parameters = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'parameters.pkl'), 'rb'))
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
try:
    epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
    best_checkpoint = checkpoint_files[epochs.index(np.argmin(val_loss) + 1)]
except:
    best_checkpoint = checkpoint_files[-1]
try:
    model = keras.models.load_model(best_checkpoint)
except:
    if pooling_type == 'downsample':
        custom_objects = {'DownSampling1D': DownSampling1D}
    elif pooling_type == 'spectral':
        custom_objects = {'SpectralPooling': SpectralPooling}
    elif pooling_type == 'argmax':
        custom_objects = {'MaxPooling1DWithArgmax': MaxPooling1DWithArgmax}
    with keras.utils.custom_object_scope(custom_objects):
        model = keras.models.load_model(best_checkpoint)
if pooling_type == 'argmax':
    for layer in model.layers:
        if isinstance(layer, MaxPooling1DWithArgmax):
            print(f'Setting store_argmax = True for layer "{layer.name}".')
            layer.store_argmax = True
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
try:
    x_train_min = network_parameters['x_train_min']
    x_train_max = network_parameters['x_train_max']
except:
    pass
var_names = network_parameters['var_names']
print(f'Loaded network from {best_checkpoint}.')
print(f'Variable names: {var_names}')

#### Plot the model topology

In [ ]:
keras.utils.plot_model(model, show_shapes=True, dpi=96)

In [ ]:
model.summary()

#### Load the data set

In [ ]:
set_name = 'test'
use_fft = network_parameters['use_fft'] if 'use_fft' in network_parameters else False
data_dirs = []
for area_ID,data_dir in zip(network_parameters['area_IDs'], network_parameters['data_dirs']):
    data_dirs.append(data_dir.format(area_ID))
data_dir = os.path.join('..', data_dirs[0])
data_files = sorted(glob.glob(data_dir + os.path.sep + f'*_{set_name}_set.h5'))
ret = load_data_areas({set_name: data_files}, network_parameters['var_names'],
                        network_parameters['generators_areas_map'][:1],
                        network_parameters['generators_Pnom'],
                        network_parameters['area_measure'],
                        trial_dur=network_parameters['trial_duration'],
                        max_block_size=100,
                        use_tf=False, add_omega_ref=True,
                        use_fft=use_fft)
y = ret[2][set_name]
if use_fft:
    X = [(ret[1][set_name][i] - m) / (M - m) for i,(m,M) in enumerate(zip(x_train_min, x_train_max))]
    F = ret[0]
else:
    X = [(ret[1][set_name][i] - m) / s for i,(m,s) in enumerate(zip(x_train_mean, x_train_std))]
    t = ret[0]

#### Predict the momentum using the model

In [ ]:
idx = [np.where(y == mom)[0] for mom in np.unique(y)]
n_mom_values = len(idx)
momentum = [np.squeeze(model.predict(X[0][jdx])) for jdx in idx]
mean_momentum = [m.mean() for m in momentum]
stddev_momentum = [m.std() for m in momentum]
print('Mean momentum:', mean_momentum)
print(' Std momentum:', stddev_momentum)

In [ ]:
cmap = plt.get_cmap('tab10', n_mom_values)
if use_fft:
    Xf = X
else:
    data_files_training = sorted(glob.glob(data_dir + os.path.sep + f'*_training_set.h5'))
    ret_fft = load_data_areas({'training': data_files_training}, network_parameters['var_names'],
                        network_parameters['generators_areas_map'][:1],
                        network_parameters['generators_Pnom'],
                        network_parameters['area_measure'],
                        trial_dur=network_parameters['trial_duration'],
                        max_block_size=200,
                        use_tf=False, add_omega_ref=True,
                        use_fft=True)
    x_train_min_fft = np.array([val.min() for val in ret_fft[1]['training']], dtype=np.float32)
    x_train_max_fft = np.array([val.max() for val in ret_fft[1]['training']], dtype=np.float32)
    ret = load_data_areas({set_name: data_files}, network_parameters['var_names'],
                        network_parameters['generators_areas_map'][:1],
                        network_parameters['generators_Pnom'],
                        network_parameters['area_measure'],
                        trial_dur=network_parameters['trial_duration'],
                        max_block_size=100,
                        use_tf=False, add_omega_ref=True,
                        use_fft=True)
    F = ret[0]
    Xf = [(ret[1][set_name][i] - m) / (M - m) for i,(m,M) in enumerate(zip(x_train_min_fft,
                                                                           x_train_max_fft))]

fig,ax = plt.subplots(1, 1, figsize=(6,4))
for i in range(len(idx)):
    mean = Xf[0][idx[i]].mean(axis=0)
    stddev = Xf[0][idx[i]].std(axis=0)
    ci = 1.96 * stddev / np.sqrt(idx[i].size)
    ax.fill_between(F, mean + ci, mean - ci, color=cmap(i))
ax.set_xscale('log')
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.grid(which='major', axis='both', ls=':', lw=0.5, color=[.6,.6,.6])
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Normalized FFT')
fig.tight_layout()
fig.savefig(f'spectra_{n_mom_values}_momentum_levels.pdf')

#### Make as many submodels as there are layers

In [ ]:
N_layers = len(model.layers)
N_vars = len(var_names)
submodels = [
    keras.Model(inputs=model.inputs, outputs=[model.layers[j].output for j in range(i,i+N_vars)])
     for i in range(N_vars, N_layers - N_vars - 3, N_vars)
]
for layer in model.layers[-4:]:
    submodels.append(keras.Model(inputs=model.inputs, outputs=layer.output))

In [ ]:
layer_id = 1
submodel = submodels[layer_id]
submodel.summary()
Y = [submodel.predict(X[0][jdx]) for jdx in idx]
print(Y[0].shape)

### Plot the inputs

In [ ]:
fig,ax = plt.subplots(figsize=(8,5))
for j,jdx in enumerate(idx):
    mean = X[0][jdx].mean(axis=0)
    stddev = X[0][jdx].std(axis=0)
    ci = 1.96 * stddev / np.sqrt(jdx.size)
    ax.fill_between(t, mean + ci, mean - ci, color=cmap(j))
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.set_xlabel('Time [min]')
ax.grid(which='major', ls=':', lw=0.5, color=[.6,.6,.6])
fig.tight_layout()

In [ ]:
# which trial to use
trial = 0
# the input to the model
x = X[0][trial:trial+1, :]
# the output of the model
y = submodel(x)
y = np.squeeze(y.numpy().T)
# let's build the time vector
t_out = t[np.newaxis,:]
for layer in submodel.layers:
    if isinstance(layer, keras.layers.Conv1D):
        kernel_size = layer.kernel_size[0]
        n_filters = layer.filters
#         if t_out.shape[0] != n_filters:
        t_out = np.tile(t_out[:,kernel_size-1:], [n_filters,1])
    elif isinstance(layer, MaxPooling1DWithArgmax):
        jdx = tf.squeeze(layer.argmax).numpy().T // kernel_size
        t_out = np.array([t_out[k,kdx] for k,kdx in enumerate(jdx)])

In [ ]:
yf = np.array([lombscargle(t_out[i], y[i], F[1:]) for i in range(t_out.shape[0])])

In [ ]:
%matplotlib notebook
cmap2 = plt.get_cmap('viridis', y.shape[0])
fig,ax = plt.subplots(1, 1, figsize=(8,5))
ax.plot(t, np.squeeze(x), 'k', lw=1)
for i,(a,b) in enumerate(zip(t_out, y)):
    ax.plot(a, b, markersize=2, lw=1, color=cmap2(i))
ax.set_xlabel('Time [min]')
ax.grid(which='major', axis='x', ls=':', lw=0.5, color=[.6,.6,.6])

In [ ]:
fig,ax = plt.subplots(1, 1)
for i in range(yf.shape[0]):
    ax.loglog(F[1:], yf[i,:], color=cmap2(i), lw=1)
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.grid(which='major', axis='both', ls=':', lw=0.5, color=[.6,.6,.6])
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Periodogram')
fig.tight_layout()

In [ ]:
n_traces, n_samples, n_kernels = y[0].shape
rows, cols = n_kernels // 8, 8
fig,ax = plt.subplots(rows, cols, figsize=(cols*1.5, rows),
                      sharex=True, sharey=False)
show_fft = False
hp_filter = False
if use_fft:
    n = -1
    m = int(n_samples / 3)
    show_fft = False
else:
    n = -1
    m = n_samples
if show_fft:
#     m = int(n_samples / 3)
    m = n_samples // 2
    Yf = []
    dt = np.diff(t[:2])[0]
    filter_order = 10
    cutoff = 0.1
    b,a = butter(filter_order//2, cutoff, 'hp', fs=1/dt)
    for y in Y:
        n_samples = y.shape[1]
        if hp_filter:
            tmp = filtfilt(b, a, y, axis=1)
        else:
            tmp = y
        tmp = fft(tmp, axis=1)
        Yf.append(2.0 / n_samples * np.abs(tmp[:, :n_samples//2, :]))    
    freq = fftfreq(n_samples, dt)[:n_samples//2]
for i in range(rows):
    for j in range(cols):
        k = i * cols + j
        if n > 0:
            for q in range(len(Y)):
                ax[i][j].plot(Y[q][:n, :m, k].T, color=cmap(q), lw=0.5)
        else:
            for q in range(len(Y)):
                if show_fft:
                    mean = Yf[q][:, :, k].mean(axis=0)
                    stddev = Yf[q][:, :, k].std(axis=0)
                    ci = 1.96 * stddev / np.sqrt(yf[q].shape[0])
                    ax[i][j].fill_between(freq, mean[:m] + ci[:m], mean[:m] - ci[:m], color=cmap(q))
                else:
                    mean = Y[q][:, :, k].mean(axis=0)
                    stddev = Y[q][:, :, k].std(axis=0)
                    ci = 1.96 * stddev / np.sqrt(y[q].shape[0])
#                     ax[i][j].fill_between(np.arange(m), mean[:m] + ci[:m], mean[:m] - ci[:m], color=cmap(q))
                    ax[i][j].fill_between(t_out, mean + ci, mean - ci, color=cmap(q))
                    ax[i][j].set_xticks(np.linspace(0, 60, 5))
#         if show_fft:
#             ax[i][j].set_xscale('log')
#         else:
#             ax[i][j].set_xticks([0, m])
        for side in 'right','top':
            ax[i][j].spines[side].set_visible(False)
        ax[i][j].set_yticklabels([])
fig.tight_layout(pad=0)
output_file = experiment_ID[:6] + f'_layer_{layer_id}_outputs'
if show_fft:
    output_file += '_spectra'
fig.savefig(output_file + '.pdf')

In [ ]:
weights = np.squeeze(model.layers[layer_id].weights[0].numpy())
print('weights shape:', weights.shape)
if len(weights.shape) > 2:
    input_id = 0
    weights = np.squeeze(weights[:, input_id, :])
else:
    input_id = 0
kernel_size, n_kernels = weights.shape
if show_fft:
    weightsf = fft(weights, axis=0)
    weightsf = 2.0 / kernel_size * np.abs(weightsf[:kernel_size//2, :])
rows, cols = n_kernels // 8, 8
fig,ax = plt.subplots(rows, cols, figsize=(cols*2, rows*1.5), sharex=True, sharey=True)
for i in range(rows):
    for j in range(cols):
        k = i * 8 + j
        if show_fft:
            ax[i][j].plot(weightsf[:,k], 'k', lw=1)
            ax[i][j].set_xticks([0, kernel_size//2])
        else:
            ax[i][j].plot(weights[:,k], 'k', lw=1)
            ax[i][j].set_xticks([0, kernel_size])
        for side in 'right','top':
            ax[i][j].spines[side].set_visible(False)
fig.tight_layout(pad=0)
output_file = experiment_ID[:6] + f'_layer_{layer_id}_input_{input_id}_weights'
if show_fft:
    output_file += '_spectra'
fig.savefig(output_file + '.pdf')

In [ ]:
raise Exception('Stop here')

## Continue from here

In [ ]:
N_blocks = len(data_sliding[0][var_names[0]])
inputs_to_network = []
inputs_to_classifier = []
outputs = []
for data_slid in data_sliding:
    for i in range(N_blocks):
        x = {var_name: tf.constant(data_slid[var_name][i:i+1,:], dtype=tf.float32) for var_name in var_names}
        inputs_to_network.append(np.concatenate([np.squeeze(data_slid[var_name][i:i+1,:]) for var_name in var_names]))
        inputs_to_classifier.append(np.squeeze(submodels[-3].predict(x)))
#         inputs_to_classifier.append(np.squeeze(submodels[-2].predict(x)))
        outputs.append(np.squeeze(submodels[-1].predict(x)))
inputs_to_network = np.array(inputs_to_network)
inputs_to_classifier = np.array(inputs_to_classifier)
outputs = np.array(outputs)
scaled_inputs_to_network = StandardScaler().fit_transform(inputs_to_network)
scaled_inputs_to_classifier = StandardScaler().fit_transform(inputs_to_classifier)

In [ ]:
area_inertia

In [ ]:
col = 'kr'
fig,ax = plt.subplots(2, 1, figsize=(10,5), sharex=False)
j = 10
for i in range(2):
    ax[0].plot(inputs_to_network[i*N_blocks + j], col[i], lw=1)
    ax[1].plot(inputs_to_classifier[i*N_blocks + j], col[i], lw=1, label=f'{outputs[i*N_blocks+j,0]:.2f}')
ax[1].legend(loc='best')

In [ ]:
inputs_to_classifier[i + j].sum()

In [ ]:
inputs_to_classifier[i*N_blocks + j].sum()

In [ ]:
dst_inputs_to_network = np.zeros(N_blocks)
dst_inputs_to_classifier = np.zeros(N_blocks)
for i in range(N_blocks):
    alignment = dtw.dtw(inputs_to_network[i], inputs_to_network[i+N_blocks], keep_internals=False)
    dst_inputs_to_network[i] = alignment.distance
    alignment = dtw.dtw(inputs_to_classifier[i], inputs_to_classifier[i+N_blocks], keep_internals=False)
    dst_inputs_to_classifier[i] = alignment.distance

In [ ]:
np.percentile(dst_inputs_to_network, [0.25,0.5,0.75])

In [ ]:
np.percentile(dst_inputs_to_classifier, [0.25,0.5,0.75])

In [ ]:
i = 0
m = np.min([inputs_to_network[i].min(), inputs_to_network[i+N_blocks].min()])
M = np.max([inputs_to_network[i].max(), inputs_to_network[i+N_blocks].max()])
dst_inputs_to_network = np.sqrt((inputs_to_network[i] - inputs_to_network[i+N_blocks]) ** 2) / (M - m)
m = np.min([inputs_to_classifier[i].min(), inputs_to_classifier[i+N_blocks].min()])
M = np.max([inputs_to_classifier[i].max(), inputs_to_classifier[i+N_blocks].max()])
dst_inputs_to_classifier = np.sqrt((inputs_to_classifier[i] - inputs_to_classifier[i+N_blocks]) ** 2) / (M - m)

plt.plot(dst_inputs_to_network, 'k', lw=0.5)
plt.plot(dst_inputs_to_classifier, 'r', lw=0.5)

In [ ]:
np.sqrt(np.sum((scaled_inputs_to_network[0] - scaled_inputs_to_network[N]) ** 2))

In [ ]:
np.sqrt(np.sum((scaled_inputs_to_classifier[0] - scaled_inputs_to_classifier[N]) ** 2))

In [ ]:
pca = PCA(n_components = 2)
pc_inputs_to_network = pca.fit_transform(scaled_inputs_to_network)
pc_inputs_to_classifier = pca.fit_transform(scaled_inputs_to_classifier)

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(7,4))
ax[0].plot(pc_inputs_to_network[:N,0], pc_inputs_to_network[:N,1], 'k.')
ax[0].plot(pc_inputs_to_network[N:,0], pc_inputs_to_network[N:,1], 'r.')
ax[1].plot(pc_inputs_to_classifier[:N,0], pc_inputs_to_classifier[:N,1], 'k.')
ax[1].plot(pc_inputs_to_classifier[N:,0], pc_inputs_to_classifier[N:,1], 'r.')

In [ ]:
embedding_inputs_to_network = umap.UMAP().fit_transform(scaled_inputs_to_network)
embedding_inputs_to_classifier = umap.UMAP().fit_transform(scaled_inputs_to_classifier)

In [ ]:
plt.plot(embedding_inputs_to_network[:N,0], embedding_inputs_to_network[:N,1],
         'ko', ms=4, markerfacecolor='w')
plt.plot(embedding_inputs_to_network[N:,0], embedding_inputs_to_network[N:,1],
         'ks', ms=4)
plt.plot(embedding_inputs_to_classifier[:N,0], embedding_inputs_to_classifier[:N,1],
         'ro', ms=4, markerfacecolor='w')
plt.plot(embedding_inputs_to_classifier[N:,0], embedding_inputs_to_classifier[N:,1],
         'rs', ms=4)

In [ ]:
output = outputs[0]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=True)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[1]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[2]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[3]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[4]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[5]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(10,3))
for i,a in enumerate(ax):
    a.plot(outputs[6+i][0], 'k', lw=1)
    a.set_xticks([])
    a.set_yticks([])
    for side in 'right','top':
        a.spines[side].set_visible(False)
fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(10,3))
for i,a in enumerate(ax):
    a.plot(outputs[6+i][0], 'k', lw=1)
    a.set_xticks([])
    a.set_yticks([])
    for side in 'right','top':
        a.spines[side].set_visible(False)
fig.tight_layout()